<a href="https://colab.research.google.com/github/ykitaguchi77/Colab_Scripts/blob/master/CoreML%E3%82%92%E7%94%A8%E3%81%84%E3%81%A6MNIST%E3%82%92ios%E3%81%ABdeploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Conversion of Pytorch script to CoreML**

既存のモデルを変換する場合<br>
https://coremltools.readme.io/docs/pytorch-conversion

In [ ]:
import torch
import torchvision
!pip install coremltools
import coremltools as ct


# Load a pre-trained version of MobileNetV2
torch_model = torchvision.models.mobilenet_v2(pretrained=True)

# Set the model in evaluation mode
torch_model.eval()

# Trace with random data
example_input = torch.rand(1, 3, 224, 224) # after test, will get 'size mismatch' error message with size 256x256
traced_model = torch.jit.trace(torch_model, example_input)


# Download class labels (from a separate file)
import urllib
label_url = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
class_labels = urllib.request.urlopen(label_url).read().decode("utf-8").splitlines()
class_labels = class_labels[1:] # remove the first class which is background
assert len(class_labels) == 1000



# Convert to Core ML using the Unified Conversion API
mlmmodel = ct.convert(
    traced_model,
    inputs=[ct.ImageType(name="input_1", shape=example_input.shape)], #name "input_1" is used in 'quickstart'
    classifier_config = ct.ClassifierConfig(class_labels) # provide only if step 2 was performed
)

# Save model
mlmodel.save("MobileNetV2.mlmodel")

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


Translating MIL ==> MLModel Ops: 100%|██████████| 704/704 [00:00<00:00, 1233.26 ops/s]


In [ ]:
from torchsummary import summary
torch_model.to(device)
summary(torch_model, (3, 224, 224))

新しく学習を行う場合（MNISTの例）<br>
参考サイト：<br>
https://qiita.com/shu223/items/6ddfbedb4fdfb2059a11<br>
https://chemicalfactory.hatenablog.com/entry/2020/01/26/230114

In [12]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


batch_size = 10

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=0)

testset = torchvision.datasets.MNIST(root='./data', train=False, 
                                        download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                            shuffle=False, num_workers=0)

class MNIST_Conv_MN(nn.Module):
    def __init__(self):
        super(MNIST_Conv_MN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3) 
        self.pooling = nn.MaxPool2d(2, 2) 
        self.fc1 = nn.Linear(13 * 13 * 8, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pooling(x)
        x = x.view(-1, 13 * 13 * 8)
        x = self.fc1(x)
        return x
    
model=MNIST_Conv_MN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

num_epochs = 10

for epoch in range(num_epochs):
    
    train_loss = 0
    train_acc = 0
    val_loss = 0
    val_acc = 0
    
    model.train()
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        train_loss += loss.item()
        train_acc += (outputs.max(1)[1] == labels).sum().item()
        loss.backward()
        optimizer.step()
    avg_train_loss = train_loss / len(trainloader.dataset)
    avg_train_acc = train_acc / len(trainloader.dataset)

    model.eval()
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_acc += (outputs.max(1)[1] == labels).sum().item()
        avg_val_loss = val_loss / len(testloader.dataset)
        avg_val_acc = val_acc / len(testloader.dataset)
        
        print("Epoch [{}/{}], Loss: {loss:.4f}, val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}"\
              .format(epoch+1, num_epochs, i+1, loss=avg_train_loss, val_loss=avg_val_loss, val_acc=avg_val_acc))
        
torch.save(model.state_dict(), "MNIST.pth")

Epoch [1/10], Loss: 0.0273, val_loss: 0.0136, val_acc: 0.9591
Epoch [2/10], Loss: 0.0105, val_loss: 0.0081, val_acc: 0.9739
Epoch [3/10], Loss: 0.0073, val_loss: 0.0073, val_acc: 0.9762
Epoch [4/10], Loss: 0.0060, val_loss: 0.0063, val_acc: 0.9807
Epoch [5/10], Loss: 0.0052, val_loss: 0.0059, val_acc: 0.9817
Epoch [6/10], Loss: 0.0046, val_loss: 0.0061, val_acc: 0.9802
Epoch [7/10], Loss: 0.0042, val_loss: 0.0067, val_acc: 0.9786
Epoch [8/10], Loss: 0.0038, val_loss: 0.0061, val_acc: 0.9789
Epoch [9/10], Loss: 0.0035, val_loss: 0.0065, val_acc: 0.9803
Epoch [10/10], Loss: 0.0031, val_loss: 0.0073, val_acc: 0.9801


In [23]:
#import coreML ver4.0
!pip install -U coremltools
import coremltools as ct


class MNIST_Conv_MN(nn.Module):
    def __init__(self):
        super(MNIST_Conv_MN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3) 
        self.pooling = nn.MaxPool2d(2, 2) 
        self.fc1 = nn.Linear(13 * 13 * 8, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pooling(x)
        x = x.view(-1, 13 * 13 * 8)
        x = self.fc1(x)
        return x
    
model = MNIST_Conv_MN()
model.load_state_dict(torch.load('MNIST.pth'))
model.eval()

scripted_model = torch.jit.script(model)

#サイズの合ったランダムな入力を行う
example_input = torch.rand(1,1,28,28)
traced = torch.jit.trace(model, example_input)

#class_labelの設定
class_labels = list(range(10))

#traced.save("model.pt")
mlmodel = ct.convert(
    traced, 
    inputs=[ct.ImageType(name="__input", shape=example_input.shape)],
    classifier_config = ct.ClassifierConfig(class_labels) ,
)
mlmodel.save('MNIST.mlmodel')

Requirement already up-to-date: coremltools in /usr/local/lib/python3.7/dist-packages (4.1)


Translating MIL ==> MLModel Ops: 100%|██████████| 19/19 [00:00<00:00, 2975.91 ops/s]


#**サンプル文字をダウンロード**

In [24]:
import numpy as np
from PIL import Image
from tensorflow.keras.datasets import mnist
import os
from google.colab import files

"""
#google driveをcolabolatoryにマウント
from google.colab import drive
drive.mount('/content/drive')

sample_dir = "/content/drive/MyDrive/Deep_learning/MNIST_samples"
"""

sample_dir = "/content/MNIST_samples"
n_samples = 30

#ダウンロード用のフォルダを作成

os.makedirs(sample_dir, exist_ok=True)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# 画像データ
# print(x_train[0])
# ラベルの値
for i in range(n_samples):
  print(y_train[i])
  img = Image.fromarray(np.uint8(x_train[i]))
  img.save(sample_dir +'/' + str(i) +'.png')

11493376/11490434 [==============================] - 0s 0us/step
5
0
4
1
9
2
1
3
1
4
3
5
3
6
1
7
2
8
6
9
4
0
9
1
1
2
4
3
2
7


#**サンプル画像でテスト推論**

In [27]:
from PIL import Image

# 推論して mlmodel に正しく変換できたか確認
# https://coremltools.readme.io/docs/model-prediction
test_image = Image.open("/content/MNIST_samples/1.png").resize((28, 28))
data = {'__input': test_image}
out_dict = model.predict(data)
print(out_dict['classLabel'])

AttributeError: ignored